# napari gui for manual matchings
to use this, we need to make sure the data files (tiff images) are organized in the way we want!  

this notebook therefore does two things:

1. load the data and arrange in the way we want -- this is necessary for us to load the channles correctly
2. launch napari to do the manual registration and save the results.

for how to load the results and then stitch the original images, that will be in the other notebook

# 0. setup the environment and define the functinos needed

In [1]:
from datetime import date
import tifffile
import napari
import os 
import glob
import pprint
import numpy as np
import pickle
# import helpers
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
import scipy.ndimage as ndi
from skimage.transform import warp
from scipy.spatial import distance
import math

from datetime import date
today = date.today()
todaystamp = today.strftime("%m%d%y")
print(todaystamp)

def wahba(X,Y):
    '''
    from X to Y transform
    '''
    X0=X-np.mean(X,axis=0)    
    Y0=Y-np.mean(Y,axis=0)    
    U, _, Vt = np.linalg.svd(X0.T@Y0)
    V = Vt.T
    M = np.eye(2)
    M[-1,:] = np.array((0,np.linalg.det(U)*np.linalg.det(V)))
    R = U@M@V.T
    T = np.mean(Y-X@R, axis = 0)
    return(np.r_[R,T[None,:]])  # return 4x3 matrix (to transform from first input to second inpupt)

100323


# 1. set up the directory. 
`file_dir` example:
```
!ls -lt /Users/shuonan/Dropbox/project/multimodal_2023/xinxindata/G126_data/tifs/ 

total 0
drwx------@ 6 shuonanchen  staff  192 Feb 18 19:00 MERFISH_DAPI
drwx------@ 6 shuonanchen  staff  192 Feb 18 18:59 MERFISH_GCaMP
drwx------@ 6 shuonanchen  staff  192 Feb 18 18:59 MERFISH_polyA
drwx------@ 6 shuonanchen  staff  192 Feb 17 16:13 MERFISH_tdTomato_G126
drwx------@ 7 shuonanchen  staff  224 Feb 17 16:13 MERFISH_BV_G126
```

In [3]:
# this is where the results will be saved. 
base_dir = '/Users/shuonan/Dropbox/project/multimodal_2023/xinxindata/G126_data/napari_rez/'

# this is where the files are located. 
file_dir = '/Users/shuonan/Dropbox/project/multimodal_2023/xinxindata/G126_data/tifs/'

In [4]:

all_files = glob.glob(file_dir+'/*')
channel_name = [f.split('/')[-1].split('_')[1] for f in all_files]
print(channel_name)  # these are all the available channels. 

['tdTomato', 'BV', 'DAPI', 'GCaMP', 'polyA']


In [5]:
# specify how many (or which) channels you want to load. 
C = len(channel_name) # if loading all the channels. 
C = 2  # by default lets only look at the tomato and BV

# 2. load and arrange the data

In [6]:
exvivos = []
exvivos.append(np.nan)  # this is to make it consistent -- exvivos[0] is nothing, since the section_id starts from 1.
for section_id in range(1,5):  # only looking at section 1,2,3,4. 
    all_channels_img = []
    for f in all_files[:C]:
        f_path = glob.glob(f+f'/*section{section_id}.tif*')
        assert(len(f_path)==1)
        f_path = f_path[0]
        img = tifffile.imread(f_path)
        img = img.astype(float)
        img /= img.max()
        all_channels_img.append(img)
    exvivos.append(np.stack(all_channels_img, axis = 0))

In [7]:
len(exvivos)  # this should be 5 == 4+1 (1 is the nan part, 4 is the number of sections we are looking)

5

In [8]:
# check the size of each
[m.shape for m in exvivos[1:]]  # these are (c,z,x,y) where c is the number of channel (2), z should be 8 always. 

[(2, 8, 2241, 2740),
 (2, 8, 2241, 2741),
 (2, 8, 2645, 2943),
 (2, 8, 2645, 2943)]

**okay the data is ready, now we are ready to launch the GUI.**
# 3. launch GUI
- notes
    - only look a pair at a time
    - dont forget to save (`q`)
- how to use
    1. select `section_id`: if `section_id=1`, then section 1 (red) and 2 (green) will be loaded. the larger index is always the source, mapped to the smaller one. (2 is transformed to map 1).
    2. focus on BV layer, adjust the gamma and contrast. hide the ohter channels when necessary.
    3. go to the `matched` layer, add points in both green and red. Note the number of points in green and red must be the same. 
    4. do the necessary transformation until you are bored with these layer (BV), then swtich to the tomato layer (hide BV, show tomato)
    5. you can clean the selected, or add more spots based on the tomato layer. 
    6. check the console output (from this notebook) for the necessary informations. 
    7. the saved results would contain (1) all the transformations applied; and (2) transformed image. 
- keys (in the using order)
    - `s`: scaling --> learn the scale and adjust the images 
    - `t`: transform --> rigid transformation
    - `c`: clear --> clear all selection of matched points. 
    - `d`: deformable --> apply deformable for the matched points.
    - `q`: quit --> well it doesnt quit the gui, it just save the results so dont press this unless you are done. 

In [9]:
# choose your section_id
section_id = 2


In [10]:

features = {'label': np.empty(0, dtype=int)}
all_transform = []
viewer = napari.view_image(exvivos[section_id][:,0],channel_axis=0,
        name=[f'sec {section_id}...{c}' for c in channel_name],
        colormap='red', visible = [True,True,False,False,False],)
exvivo_layer = viewer.add_image(exvivos[section_id+1][:,-1],channel_axis=0,
        name=[f'sec {section_id+1}...{c}' for c in channel_name],
        colormap="green", visible = [True,True,False,False,False],)
pl_in = viewer.add_points(
    size=5, edge_width=1, edge_color='red',face_color='transparent',    
    name=f"matched section {section_id}", text='label', features=features,ndim=2,out_of_slice_display=True)
@pl_in.events.data.connect
def update_feature_default_invivo():  
    global points_layer
    no_of_points = len(pl_in.data)
    pl_in.feature_defaults['label'] = no_of_points + 1
    pl_in.properties["label"][0:no_of_points] = range(1, no_of_points+1)
    pl_in.text.values[0:no_of_points] = [str(i) for i in range(1, no_of_points+1)]
    pl_in.text.color = 'red'
    pl_in.text.translation = np.array([-10, 0])
update_feature_default_invivo()
pl_in.mode = 'add'

pl_ex = viewer.add_points(
    size=5, edge_width=1, edge_color='green',face_color='transparent',    
    name=f"matched section {section_id+1}", text='label', features=features,ndim=2,out_of_slice_display=True)
@pl_ex.events.data.connect
def update_feature_default_exvivo():  
    global points_layer
    no_of_points = len(pl_ex.data)
    pl_ex.feature_defaults['label'] = no_of_points + 1
    pl_ex.properties["label"][0:no_of_points] = range(1, no_of_points+1)
    pl_ex.text.values[0:no_of_points] = [str(i) for i in range(1, no_of_points+1)]
    pl_ex.text.color = 'green'
    pl_ex.text.translation = np.array([-10, 0])
update_feature_default_exvivo()
pl_ex.mode = 'add'

@viewer.bind_key('s', overwrite = True)
def scale(viewer):  
    print('learning scale..')
    m_invivo = viewer.layers[f"matched section {section_id}"].data
    m_exvivo = viewer.layers[f"matched section {section_id+1}"].data 
    assert(len(m_invivo)==len(m_exvivo))    
    s_invivo = np.sqrt(np.sum((m_invivo-np.mean(m_invivo,0))**2)/len(m_invivo))
    s_exvivo = np.sqrt(np.sum((m_exvivo-np.mean(m_exvivo,0))**2)/len(m_exvivo))
    scl = s_invivo/s_exvivo
    print(f'scale difference: {scl}')
    m_exvivo_new = scl*m_exvivo    
    exvivo_scaled_small = np.array([ndi.zoom(exvivo_layer[c].data, scl, order=3) for c in range(C)])    
    all_transform.append(dict(scale=scl))
    for c in range(C):
        exvivo_layer[c].data = exvivo_scaled_small[c]
    viewer.layers[f"matched section {section_id+1}"].data = m_exvivo_new    
    

@viewer.bind_key('t', overwrite = True)
def transform(viewer):    
    print('applying rigid..')
    m_invivo = viewer.layers[f"matched section {section_id}"].data
    m_exvivo = viewer.layers[f"matched section {section_id+1}"].data
    assert(len(m_invivo)==len(m_exvivo))
    bhat = wahba(m_exvivo,m_invivo)
    offset = -(bhat[:2,:2])@bhat[-1]
    exvivo_affined = np.array([ndi.affine_transform(exvivo_layer[c].data, bhat[:2,:2],
                                                    output_shape = exvivo_layer[0].data.shape,offset = offset, order=3) for c in range(C)])
    foo = np.c_[m_exvivo, np.ones((m_exvivo.shape[0],1))]   #Nx4
    m_exvivo_update = foo@bhat
    all_transform.append(dict(bhat=bhat))
    for c in range(C):
        exvivo_layer[c].data = exvivo_affined[c]
    viewer.layers[f"matched section {section_id+1}"].data = m_exvivo_update

    
@viewer.bind_key('d', overwrite = True)
def deform(viewer):    
    print('applying deformable..')
    ksz = 200
    m_invivo = viewer.layers[f"matched section {section_id}"].data
    m_exvivo = viewer.layers[f"matched section {section_id+1}"].data
    assert(len(m_invivo)==len(m_exvivo))    
    shift = m_invivo-m_exvivo   # so the newcoords = old+shift --> vecfield will have how much shift needed to apply to the old coords
    vec_field = np.zeros(exvivo_layer[0].data.shape + (2,))  # M1,M2,2
    for p,loc in enumerate(m_exvivo):
        vec_field[int(loc[0]),int(loc[1])] = shift[p]
    for c in range(2):
        vec_field[...,c] = ndi.gaussian_filter(vec_field[...,c], ksz)
    A = np.zeros_like(m_exvivo)
    for p,loc in enumerate(m_exvivo):
        A[p] = vec_field[int(loc[0]),int(loc[1])]
    diag_step,_,_,_ = np.linalg.lstsq(A,shift,rcond=None)
    step = np.diag(diag_step)
    vec_field_total =vec_field*step;  # element wise. 
    all_transform.append(dict(vec_field_total=vec_field_total))
    mapx_base, mapy_base = np.meshgrid(np.arange(exvivo_layer[0].data.shape[0]),np.arange(exvivo_layer[0].data.shape[1]), indexing='ij')
    mapx=mapx_base-vec_field_total[:,:,0]
    mapy=mapy_base-vec_field_total[:,:,1]
    for c in range(C):
        img_de = warp(exvivo_layer[c].data,np.array((mapx,mapy)), order = 3)            
        exvivo_layer[c].data = img_de    
    
    m_exvivo_new = np.zeros_like(m_exvivo)  # POINTS
    for p,loc in enumerate(m_exvivo):
        new_s = vec_field_total[int(loc[0]),int(loc[1])]
        m_exvivo_new[p] = loc+new_s
    viewer.layers[f"matched section {section_id+1}"].data = m_exvivo_new
    
@viewer.bind_key('c')
def clear_selected(viewer):        
    viewer.layers[f"matched section {section_id}"].data = np.empty((0, 2))
    viewer.layers[f"matched section {section_id+1}"].data = np.empty((0, 2))

    
@viewer.bind_key('q')
def save_rez(viewer):     
    out_dict = dict()
    out = np.array([exvivo_layer[c].data for c in range(2)] )
    out_dict[f'transformed_section_{section_id+1}']=out
    out_dict['transformations']=all_transform    
    pickle.dump(out_dict, open(base_dir + f'section_{section_id}_{section_id+1}_{todaystamp}.pkl','wb'))
    print("saved the results!")
    

applying rigid..
learning scale..
scale difference: 0.9982306505448446
applying rigid..
learning scale..
scale difference: 1.0020945622556776
applying rigid..
applying rigid..
applying deformable..


In [11]:
todaystamp

'022123'